# Importing necessary libraries

In [1]:
import yfinance as yf
import psycopg2
import holidays
import pandas as pd
from datetime import timedelta

# Connect to PostgreSQL database

In [2]:
def connect_to_db():
    try:
        conn = psycopg2.connect(
            host="localhost",  
            database="stock_data",  
            user="umutknsk",  
            password="umut1234"  
        )
        cur = conn.cursor()
        return conn, cur
    except Exception as e:
        print(f"Error connecting to database: {e}")

# Function to analyze missing dates and print results

In [ ]:
def analyze_missing_dates(symbol, first_date, last_date, conn, cur):  
    us_holidays = holidays.US(years=range(first_date.year, last_date.year + 1))

    all_trading_days = [i.date() for i in pd.date_range(start = first_date, end = last_date) 
                        if i.weekday() < 5 and i.date() not in us_holidays]

    cur.execute("""SELECT DISTINCT timestamp FROM stock_table WHERE symbol = %s;""", (symbol,))
    existing_dates = {row[0].date() for row in cur.fetchall()}

    missing_dates = sorted(set(all_trading_days) - existing_dates)

    missing_dates = [date for date in missing_dates if date not in us_holidays]

    print(f"For {symbol}, missing dates are: {missing_dates}")
    print("\n")

# Main Process

In [4]:
conn, cur = connect_to_db()

symbols = ['AAPL', 'NVDA', 'SOUN', 'SMCI']

for symbol in symbols:
    stock = yf.Ticker(symbol)
    stock_info = stock.history(period='max')

    if stock_info.empty:
        print(f"No stock data available for {symbol}.")
        continue
    
    stock_info.index = stock_info.index.tz_localize(None)
    first_date = stock_info.index.min()  
    last_date = stock_info.index.max()
      
    analyze_missing_dates(symbol, first_date, last_date, conn, cur)

cur.close()
conn.close()

For AAPL, missing dates are: [datetime.date(1981, 4, 17), datetime.date(1982, 1, 25), datetime.date(1982, 4, 9), datetime.date(1983, 4, 1), datetime.date(1984, 4, 20), datetime.date(1985, 4, 5), datetime.date(1985, 7, 18), datetime.date(1985, 9, 27), datetime.date(1985, 12, 10), datetime.date(1986, 3, 28), datetime.date(1987, 3, 17), datetime.date(1987, 4, 17), datetime.date(1988, 4, 1), datetime.date(1989, 3, 24), datetime.date(1990, 4, 13), datetime.date(1990, 12, 17), datetime.date(1991, 3, 29), datetime.date(1992, 4, 17), datetime.date(1993, 4, 9), datetime.date(1993, 4, 14), datetime.date(1994, 4, 1), datetime.date(1994, 4, 27), datetime.date(1995, 4, 14), datetime.date(1996, 4, 5), datetime.date(1997, 3, 28), datetime.date(1998, 4, 10), datetime.date(1999, 4, 2), datetime.date(2000, 4, 21), datetime.date(2001, 4, 13), datetime.date(2001, 9, 11), datetime.date(2001, 9, 12), datetime.date(2001, 9, 13), datetime.date(2001, 9, 14), datetime.date(2002, 3, 29), datetime.date(2003, 4, 1